# Solving 3-SAT Problem with Hybrid Quantum-Classical solver 

**Author:** Xinghui Jia \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[Janus-CT: A Hybrid Approach for 3-SAT Problems by Integrating Quantum Annealer with CDCL][1]"

[1]: https://ieeexplore.ieee.org/abstract/document/10071022/


A propositional satisfiability (SAT) problem is to find an assignment for each variable to satisfy a given Boolean formula. A typical SAT problem is composed of multiple clauses represented as a disjunction of Boolean variables. 3-SAT problem is a special case of the SAT problem that has no more than three variables in each clause. 

3-SAT problem is a fundamental problem in various applications, such as artificial intelligence, circuit analysis, and protein structure prediction. Since the 3-SAT problem is an NP-complete problem, the time complexity of classical algorithms increases exponentially with the number of clauses and variables. We observe that the classical CDCL algorithm has the advantage of solving larger problems, while QA is more suitable for solving small but hard problems. To combine their advantages, we introduce Janus-SAT in this notebook, a hybrid approach that integrates quantum annealing (QA) with the classical Conflict-Driven Clause Learning (CDCL) algorithm to enable end-to-end acceleration for solving 3-SAT problems. 


<div style="text-align:center;">
    <img src="./pictures/4-1.hyqsat.png"  width="45%" height="45%">
</div>

The figure above shows the workflow of Janus-CT. It features a cross-iterative process. The CDCL algorithm searches for the solution. During the search, it identifies and sends the hard sub-problem to QA. Janus-CT designed a fast compilation flow to embed the problem and optimize the noise. It also designs multiple strategies to guide the search of CDCL using the QA results.

In [1]:
import sys
import os
import logging

sys.path.append('..')

from janusq.hyqsat import solve_by_janusct, solve_by_minisat

In [2]:
# Configure the model
file_path = "cnf_examples/UF100/uf100-01.cnf"  # input cnf flie
cpu_lim = 0  # limit the cpu time (s). 0 means infinite
mem_lim = 0  # limit the memory 0 means infinite
strictp = True  # isStrict

### Using Janus-SAT API to Solve the Problem

We can use a QA simulator or real-world QA hardware to solve the SAT roblem. For example, here we use the simualtor.

In [3]:
# solve by Janus-SAT
solve_by_janusct(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 1,
 'conflicts': 7,
 'conflict cost': 0.067,
 'decisions': 4,
 'decisions cost': 0.032,
 'propagations': 325,
 'propagations cost': 0.305,
 'conflict literals': 50,
 'actual CPU time': 7672.84,
 'solving time': 4.035,
 'annealing time': 3.48,
 'simulation time': 1312.69,
 'quantum count': 29,
 'quantum success number': 11,
 'quantum conflict number': 18,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

Note that when using the simulator, the solving time is estimated as (CDCL time + number of QA $\times$ 120 $\mu s$).

### Comparision to MiniSAT

We compare it to the MiniSAT solver.

In [4]:
# solve by minisat
solve_by_minisat(file_path, verb=False, cpu_lim=cpu_lim, mem_lim=mem_lim)

{'restarts': 3,
 'conflicts': 384,
 'conflict cost': 2.588,
 'decisions': 463,
 'decisions cost': 0.436,
 'propagations': 9130,
 'propagations cost': 5.649,
 'conflict literals': 2416,
 'actual CPU time': 102.369,
 'solving time': 10.346,
 'annealing time': 0.0,
 'simulation time': 0.0,
 'quantum count': 0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'is satisfiable': True,
 'is sat': True}

In [5]:
all_result = {}
dir_name = 'cnf_examples/test'
for filename in os.listdir(dir_name):
    janusq_res = solve_by_janusct(os.path.join(dir_name, filename), verb=False)
    minisat_res = solve_by_minisat(os.path.join(dir_name, filename), verb=False)
    for key in janusq_res:
        if key not in all_result:
            all_result[key] = {
                'janus': 0,
                'minisat': 0
            }
        all_result[key]['janus'] += janusq_res[key]
        all_result[key]['minisat'] += minisat_res[key]

print(all_result)    

{'restarts': {'janus': 12, 'minisat': 19}, 'conflicts': {'janus': 130, 'minisat': 1383}, 'conflict cost': {'janus': 0.9600000000000001, 'minisat': 9.043000000000001}, 'decisions': {'janus': 38, 'minisat': 1814}, 'decisions cost': {'janus': 0.29800000000000004, 'minisat': 1.7349999999999999}, 'propagations': {'janus': 3986, 'minisat': 32252}, 'propagations cost': {'janus': 3.468, 'minisat': 19.529999999999998}, 'conflict literals': {'janus': 782, 'minisat': 8726}, 'actual CPU time': {'janus': 88728.0, 'minisat': 852.031}, 'solving time': {'janus': 46.221000000000004, 'minisat': 36.023}, 'annealing time': {'janus': 39.720000000000006, 'minisat': 0.0}, 'simulation time': {'janus': 14152.242, 'minisat': 0.0}, 'quantum count': {'janus': 331, 'minisat': 0}, 'quantum success number': {'janus': 136, 'minisat': 0}, 'quantum conflict number': {'janus': 195, 'minisat': 0}, 'quantum one time solve number': {'janus': 0, 'minisat': 0}, 'is satisfiable': {'janus': 12, 'minisat': 12}, 'is sat': {'janu

In [6]:
import pandas as pd

df = pd.DataFrame(all_result)
df

,restarts,conflicts,conflict cost,decisions,decisions cost,propagations,propagations cost,conflict literals,actual CPU time,solving time,annealing time,simulation time,quantum count,quantum success number,quantum conflict number,quantum one time solve number,is satisfiable,is sat
janus,12,130,0.960,38,0.298,3986,3.468,782,88728.000,46.221,39.72,14152.242,331,136,195,0,12,12
minisat,19,1383,9.043,1814,1.735,32252,19.530,8726,852.031,36.023,0.00,0.000,0,0,0,0,12,12
